In [1]:
# !wget https://raw.githubusercontent.com/huseinzol05/malay-dataset/master/translation/iium-confession/part1.txt
# !wget https://raw.githubusercontent.com/huseinzol05/malay-dataset/master/translation/iium-confession/part2.txt
# !wget https://raw.githubusercontent.com/huseinzol05/malay-dataset/master/translation/iium-confession/part3.txt
# !wget https://raw.githubusercontent.com/huseinzol05/malay-dataset/master/translation/iium-confession/part4.txt

In [2]:
# !cp ~/dev/malaya/misc/normalize-twitter/*.requested .
# !cp ~/iium-confession/*.requested .
# !cp ~/facebook/*.requested2 .

In [3]:
# !cp ~/dev/malaya/misc/normalize-twitter/*.ms-requested .
# !cp ~/iium-confession/*.ms-requested .
# !cp ~/facebook/*.ms-requested .

In [4]:
from glob import glob
import json
import re
import random

def cleaning(string):
    string = string.replace('\n', ' ').replace('\r', ' ')
    return re.sub(r'[ ]+', ' ', string).strip()

In [5]:
train = open('train.json', 'w')

In [6]:
iium_crowd = glob('part*.txt')
for f in iium_crowd:
    with open(f) as fopen:
        for l in fopen:
            try:
                l, r = l.split('<>')
                l = cleaning(l)
                r = cleaning(r)
                if len(l) and len(r):
                    d = {"translation": {"src": l, "tgt": r, 'prefix': 'terjemah Melayu ke Inggeris: '}}
                    train.write(f'{json.dumps(d)}\n')
                    
                    d = {"translation": {"src": r, "tgt": l, 'prefix': 'terjemah Inggeris ke pasar Melayu: '}}
                    train.write(f'{json.dumps(d)}\n')
            except Exception as e:
                # print(e)
                pass

In [7]:
with open('train-test-split.json') as fopen:
    twitter_train_test = json.load(fopen)
    twitters_test = twitter_train_test['test']
    twitters_train = twitter_train_test['train']

In [8]:
with open('test.json', 'w') as fopen_jsonl:
    for i in range(len(twitters_test)):
        en = twitters_test[i][0]
        tgt = twitters_test[i][1]
        d = {"translation": {"src": en, "tgt": tgt, 'prefix': 'terjemah Melayu ke Inggeris: '}}
        fopen_jsonl.write(f'{json.dumps(d)}\n')

In [9]:
for i in range(len(twitters_train)):
    en = twitters_train[i][0]
    tgt = twitters_train[i][1]
    d = {"translation": {"src": en, "tgt": tgt, 'prefix': 'terjemah Melayu ke Inggeris: '}}
    train.write(f'{json.dumps(d)}\n')
    
    d = {"translation": {"src": tgt, "tgt": en, 'prefix': 'terjemah Inggeris ke pasar Melayu: '}}
    train.write(f'{json.dumps(d)}\n')

In [10]:
!tail -n 3 train.json

{"translation": {"src": "Just posted a photo @ Telok Ayer Tawar, Penang, Malaysia https://t.co/rFuiLzWEZ7", "tgt": "Just posted a photo @ Telok Ayer Tawar, Pulau Pinang, Malaysia https://t.co/rFuiLzWEZ7", "prefix": "terjemah Inggeris ke pasar Melayu: "}}
{"translation": {"src": "Asal SHAH ALAM still penuh Woi", "tgt": "The origin of SHAH ALAM is still full of Woi", "prefix": "terjemah Melayu ke Inggeris: "}}
{"translation": {"src": "The origin of SHAH ALAM is still full of Woi", "tgt": "Asal SHAH ALAM still penuh Woi", "prefix": "terjemah Inggeris ke pasar Melayu: "}}


In [11]:
twitters = glob('*.normalized.jsonl.requested.ms-requested')
for f in twitters:
    with open(f) as fopen:
        for l in fopen:
            try:
                data = json.loads(l)
                en = data['src']['r']['result']
                ori = cleaning(data['src']['src']['original'])
                norm = cleaning(data['src']['src']['normalized'])
                tgt = cleaning(data['r']['result'])
                
                d = {"translation": {"src": ori, "tgt": tgt, 'prefix': 'terjemah pasar Melayu ke Melayu: '}}
                train.write(f'{json.dumps(d)}\n')
                
                d = {"translation": {"src": norm, "tgt": tgt, 'prefix': 'terjemah pasar Melayu ke Melayu: '}}
                train.write(f'{json.dumps(d)}\n')
                
                d = {"translation": {"src": tgt, "tgt": ori, 'prefix': 'terjemah Melayu ke pasar Melayu: '}}
                train.write(f'{json.dumps(d)}\n')

            except Exception as e:
                print(e)
                pass

In [12]:
!tail -n 3 train.json

{"translation": {"src": "Jim kadar bayan daukarsa baiki ia pergi dan buro di, ya aksara datang kamfani dah halinsa ia bera. Matashi ya beca dah dan ada https://t.co/w6WyfWbFkv", "tgt": "Jim kadar bayan sakunsa baiki dia pergi na buro di, iya watak datang kompeni udah halinsa dia bera. Matashi ya beca dan ada https://t.co/w6WyfWbFkv", "prefix": "terjemah pasar Melayu ke Melayu: "}}
{"translation": {"src": "Jim kadar bayan sakunsa baiki dia pergi na buro di, iya watak datang kompeni udah halinsa dia bera. Matashi ya beca dan ada https://t.co/w6WyfWbFkv", "tgt": "Jim kadan bayan daukarsa aiki a gidan burodi, ya kassara mai kamfani da halinsa na bera. Matashi ya fece da kudade https://t.co/w6WyfWbFkv", "prefix": "terjemah Melayu ke pasar Melayu: "}}
{"translation": {"src": "Dia kat lane dalam tapi nk masuk kanan", "tgt": "Dia berada di lorong dalam tetapi mahu masuk ke sebelah kanan", "prefix": "terjemah pasar Melayu ke Melayu: "}}


In [13]:
iiums = glob('clean-iium*.splitted.requested')
iiums.append('output.error.requested')

for f in iiums:
    with open(f) as fopen:
        for l in fopen:
            try:
                data = json.loads(l)
                l = cleaning(data['src'])
                r = cleaning(data['r']['result'])
                if len(l) and len(r):
                    d = {"translation": {"src": l, "tgt": r, 'prefix': 'terjemah Melayu ke Inggeris: '}}
                    train.write(f'{json.dumps(d)}\n')
                    
                    d = {"translation": {"src": r, "tgt": l, 'prefix': 'terjemah Inggeris ke pasar Melayu: '}}
                    train.write(f'{json.dumps(d)}\n')
            except Exception as e:
                pass

In [14]:
!tail -n 3 train.json

{"translation": {"src": "Jadi aku pun wasap Mak aku yang aku cintai malam tu jugak. Sambil-sambil tu layan lah makan western tu tapi.. dah x fikir dah nikmat makanan tu sbb fikir nk settle kan benda yang berserabut dalam kepala otak aku ni haa. So aku wasap mak aku betapa aku sangat x senang hidup macam ni, sebab aku memang niat datang ke kampus ini untuk belajar sebab aku berjaya dapat bidang yang minat so xkan nk sia-siakan rezeki yang Allah beri ni kan? Jadi aku pun menyusun ayat mengatakan aku ada crush pada seseorang ni, dan aku rasa aku mmg x mampu untuk go to the next level. So aku telah membuat suatu keputusan dimana aku yakin pada malam tu Allah adalah pemberi ilham yang terbaik untuk diri aku. Aku kata kat Mak aku \" Mak.. along rasa, along perlu buat keputusan ini dan along mintak maaf kalau perkara ini akan menyusahkan mak\" so mak aku pun balas pendek je.. \" ok ape dia?\"...", "tgt": "So I smoked my mom whom I love that night too. Meanwhile, I was eating the western food 

In [15]:
iiums = glob('clean-iium*.splitted.requested.ms-requested')
iiums.append('output.error.requested.ms-requested')

for f in iiums:
    with open(f) as fopen:
        for l in fopen:
            try:
                data = json.loads(l)
                l = cleaning(data['src']['src'])
                r = cleaning(data['r']['result'])
                
                if len(l) and len(r):
                    d = {"translation": {"src": l, "tgt": r, 'prefix': 'terjemah pasar Melayu ke Melayu: '}}
                    train.write(f'{json.dumps(d)}\n')
                    
                    d = {"translation": {"src": r, "tgt": l, 'prefix': 'terjemah Melayu ke pasar Melayu: '}}
                    train.write(f'{json.dumps(d)}\n')
            except Exception as e:
                pass

In [16]:
!tail -n 2 train.json

{"translation": {"src": "Jadi aku pun wasap Mak aku yang aku cintai malam tu jugak. Sambil-sambil tu layan lah makan western tu tapi.. dah x fikir dah nikmat makanan tu sbb fikir nk settle kan benda yang berserabut dalam kepala otak aku ni haa. So aku wasap mak aku betapa aku sangat x senang hidup macam ni, sebab aku memang niat datang ke kampus ini untuk belajar sebab aku berjaya dapat bidang yang minat so xkan nk sia-siakan rezeki yang Allah beri ni kan? Jadi aku pun menyusun ayat mengatakan aku ada crush pada seseorang ni, dan aku rasa aku mmg x mampu untuk go to the next level. So aku telah membuat suatu keputusan dimana aku yakin pada malam tu Allah adalah pemberi ilham yang terbaik untuk diri aku. Aku kata kat Mak aku \" Mak.. along rasa, along perlu buat keputusan ini dan along mintak maaf kalau perkara ini akan menyusahkan mak\" so mak aku pun balas pendek je.. \" ok ape dia?\"...", "tgt": "Jadi saya merokok ibu saya yang saya sayangi malam itu juga. Sambil-sambil tu tengah mak

In [17]:
facebooks = glob('*.splitted.requested2')
facebooks

['facebook-data.jsonl00.splitted.requested2',
 'facebook-data-v2.jsonl01.splitted.requested2',
 'filtered-common-crawl02.splitted.requested2',
 'facebook-data.jsonl02.splitted.requested2',
 'facebook-data.jsonl03.splitted.requested2',
 'facebook-data-v2.jsonl00.splitted.requested2',
 'facebook-data-v2.jsonl02.splitted.requested2',
 'facebook-data.jsonl01.splitted.requested2',
 'facebook-data-v2.jsonl03.splitted.requested2']

In [18]:
for f in facebooks:
    with open(f) as fopen:
        for l in fopen:
            try:
                data = json.loads(l)
                l = cleaning(data['src']['text'])
                r = cleaning(data['r']['result'])
                l_ = cleaning(data['src']['original'])
                if len(l) and len(r):
                    d = {"translation": {"src": l, "tgt": r, 'prefix': 'terjemah Melayu ke Inggeris: '}}
                    train.write(f'{json.dumps(d)}\n')
                    
                    d = {"translation": {"src": r, "tgt": l, 'prefix': 'terjemah Inggeris ke pasar Melayu: '}}
                    train.write(f'{json.dumps(d)}\n')
                    
                if len(l_) and len(r):
                    d = {"translation": {"src": l_, "tgt": r, 'prefix': 'terjemah Melayu ke Inggeris: '}}
                    train.write(f'{json.dumps(d)}\n')
                    
                    d = {"translation": {"src": r, "tgt": l_, 'prefix': 'terjemah Inggeris ke pasar Melayu: '}}
                    train.write(f'{json.dumps(d)}\n')
            except Exception as e:
                pass

In [19]:
!tail -n 2 train.json

{"translation": {"src": "GPS NATIONAL LINE", "tgt": "BN GPS", "prefix": "terjemah Inggeris ke pasar Melayu: "}}
{"translation": {"src": "Sy pun geng 2hb. lejar masih 2020", "tgt": "I'm also in the 2nd gang. ledger is still 2020", "prefix": "terjemah Melayu ke Inggeris: "}}


In [20]:
facebooks = glob('*facebook*.ms-requested')
facebooks

['facebook-data-v2.jsonl01.splitted.requested2.ms-requested',
 'facebook-data-v2.jsonl01.splitted.requested.ms-requested',
 'facebook-data-v2.jsonl02.splitted.requested2.ms-requested',
 'facebook-data.jsonl02.splitted.requested2.ms-requested',
 'facebook-data-v2.jsonl00.splitted.requested2.ms-requested',
 'facebook-data.jsonl03.splitted.requested2.ms-requested',
 'facebook-data-v2.jsonl03.splitted.requested2.ms-requested',
 'facebook-data.jsonl01.splitted.requested2.ms-requested',
 'facebook-data.jsonl00.splitted.requested2.ms-requested']

In [21]:
for f in facebooks:
    with open(f) as fopen:
        for l in fopen:
            try:
                data = json.loads(l)
                l = cleaning(data['src']['src']['original'])
                l_ = cleaning(data['src']['src']['text'])
                r = cleaning(data['r']['result'])

                if len(l) and len(r):
                    d = {"translation": {"src": l, "tgt": r, 'prefix': 'terjemah pasar Melayu ke Melayu: '}}
                    train.write(f'{json.dumps(d)}\n')
                    
                    d = {"translation": {"src": r, "tgt": l, 'prefix': 'terjemah Melayu ke pasar Melayu: '}}
                    train.write(f'{json.dumps(d)}\n')
                    
                if len(l_) and len(r):
                    d = {"translation": {"src": l_, "tgt": r, 'prefix': 'terjemah pasar Melayu ke Melayu: '}}
                    train.write(f'{json.dumps(d)}\n')
            except Exception as e:
                pass

In [22]:
!tail -n 3 train.json

{"translation": {"src": "Nasib baik 22 bulan Kalau 5 tahun dah jadi melayu Singapore", "tgt": "Nasib baik 22 bulan Kalau 5 tahun dah jadi Melayu Singapura", "prefix": "terjemah pasar Melayu ke Melayu: "}}
{"translation": {"src": "Saya x minta byk DSAi ..setiap jawatan adalah amanah .apa peekara yang boleh membawa kebaikkan ..teruskan DSAi ..lpakn mulut orang..bila kita buat kerana Allah SWT, Allah SWT akan tolong hambanya. Teruskan membuat kebaikkan .", "tgt": "Saya tak minta banyak DSAI. . setiap jawatan adalah amanah. apa yang boleh mendatangkan kebaikan. . teruskan DSAI. . apa salahnya mulut orang kalau kita buat kerana Allah SWT, Allah SWT akan bantu hambanya. teruskan berbuat baik.", "prefix": "terjemah pasar Melayu ke Melayu: "}}
{"translation": {"src": "Saya tak minta banyak DSAI. . setiap jawatan adalah amanah. apa yang boleh mendatangkan kebaikan. . teruskan DSAI. . apa salahnya mulut orang kalau kita buat kerana Allah SWT, Allah SWT akan bantu hambanya. teruskan berbuat baik.

In [26]:
common_crawls = glob('filtered-common-crawl*')

for f in common_crawls:
    with open(f) as fopen:
        for l in fopen:
            try:
                data = json.loads(l)
                l = cleaning(data['src'])
                r = cleaning(data['r']['result'])
                if len(l) and len(r):
                    d = {"translation": {"src": l, "tgt": r, 'prefix': 'terjemah Melayu ke Inggeris: '}}
                    train.write(f'{json.dumps(d)}\n')
                    
                    d = {"translation": {"src": r, "tgt": l, 'prefix': 'terjemah Inggeris ke pasar Melayu: '}}
                    train.write(f'{json.dumps(d)}\n')
            except Exception as e:
                pass

In [27]:
train.close()

In [28]:
from tqdm import tqdm

with open('filtered-train.json', 'w') as train:
    with open('train.json') as fopen:
        for l in tqdm(fopen):
            data = json.loads(l)
            src = data['translation']['src']
            tgt = data['translation']['tgt']
            if ('promo' in src and 'firm' in tgt) or ('firm' in src and 'promo' in tgt):
                continue
            train.write(f'{json.dumps(data)}\n')

8635532it [00:36, 235369.72it/s]


In [29]:
!wc -l filtered-train.json

8634891 filtered-train.json


In [30]:
!shuf filtered-train.json > shuffled-train.json

In [31]:
!shuf test.json > shuffled-test.json

In [32]:
data = []

with open('filtered-train.json') as fopen:
    for l in tqdm(fopen):
        if 'terjemah pasar Melayu ke Melayu' in l:
            data.append(l)
            
len(data)

8634891it [00:03, 2495153.17it/s]


2112709